In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 44 kB/s 
     |████████████████████████████████| 199 kB 61.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=2ccfe345dab60509b25252af6438f58426f806018c3f90273e75465738c21347
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

spark = SparkSession\
        .builder\
        .master('local[*]')\
        .getOrCreate()
sc = spark.sparkContext

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#change the directory
%cd /content/drive/My Drive/Colab Notebooks/big-data-Colab/
#pwd
!ls

/content/drive/My Drive/Colab Notebooks/big-data-Colab
 2.DataCopy.ipynb		 JosephNikhilReddy_MT_Q2_B.txt
 2.WordCount.ipynb		 JosephNikhilReddy_MT_Q3_A.txt
 4.MR-DataJoin.ipynb		 JosephNikhilReddy_MT_Q3_B.txt
 4.MR-WordCount.ipynb		 make_join2data.py
 5.MR-DataJoin2.ipynb		'Midterm Section2 Q1.ipynb'
 6.MR-WordCountReducer.ipynb	'Midterm Section2 Q2.ipynb'
 7.Spark-WordCount.ipynb	'Midterm Section2 Q3.ipynb'
 ad-clicks.csv.gz		 MT_Q1_make_data.py
 BigData.ipynb			 MT_Q2_make_data.py
 buy-clicks.csv.gz		 MT_Q3_make_data.py
 game-clicks.csv.gz		 out
 hadoop-3.3.4			 out0
 hadoop-3.3.4.tar.gz		 out2
 join1_FileA.txt		 out3
 join1_FileB.txt		 outDataJoin
 join1_mapper.py		 outDataJoin2
 join1_mapper_Q2.py		 Q1out
 join1_reducer.py		 Q2out
 join1_reducer_Q2.py		 SparkOutPut
 join2_genchanA.txt		 testfile1
 join2_genchanB.txt		 testfile2
 join2_genchanC.txt		 Untitled0.ipynb
 join2_gennumA.txt		 wordcount_mapper.py
 join2_gennumB.txt		 wordcount_mapper_Q1.py
 join2_gennumC.txt		 wordcount_nu

In [5]:
from pyspark.sql import SQLContext
sqlsc = SQLContext(sc)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


In [6]:
df =sqlsc.read.load('/content/drive/My Drive/Colab Notebooks/big-data-Colab/game-clicks.csv.gz',
                          format='com.databricks.spark.csv',
                          header = 'true',
                          inferSchema='true')

In [7]:
df.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- clickId: integer (nullable = true)
 |-- userId: integer (nullable = true)
 |-- userSessionId: integer (nullable = true)
 |-- isHit: integer (nullable = true)
 |-- teamId: integer (nullable = true)
 |-- teamLevel: integer (nullable = true)



In [8]:
df.count()

755806

In [9]:
df.show(5)

+-------------------+-------+------+-------------+-----+------+---------+
|          timestamp|clickId|userId|userSessionId|isHit|teamId|teamLevel|
+-------------------+-------+------+-------------+-----+------+---------+
|2016-05-26 15:06:55|    105|  1038|         5916|    0|    25|        1|
|2016-05-26 15:07:09|    154|  1099|         5898|    0|    44|        1|
|2016-05-26 15:07:14|    229|   899|         5757|    0|    71|        1|
|2016-05-26 15:07:14|    322|  2197|         5854|    0|    99|        1|
|2016-05-26 15:07:20|     22|  1362|         5739|    0|    13|        1|
+-------------------+-------+------+-------------+-----+------+---------+
only showing top 5 rows



In [10]:
df.select("userid","teamlevel").show(5)

+------+---------+
|userid|teamlevel|
+------+---------+
|  1038|        1|
|  1099|        1|
|   899|        1|
|  2197|        1|
|  1362|        1|
+------+---------+
only showing top 5 rows



In [11]:
df.filter(df["teamlevel"] >1).select("userid", "teamlevel").show(5)

+------+---------+
|userid|teamlevel|
+------+---------+
|  1513|        2|
|   868|        2|
|  1453|        2|
|  1282|        2|
|  1473|        2|
+------+---------+
only showing top 5 rows



In [12]:
df.groupBy("ishit").count().show()

+-----+------+
|ishit| count|
+-----+------+
|    1| 83383|
|    0|672423|
+-----+------+



In [13]:
from pyspark.sql.functions import *
df.select(mean('ishit'),sum('ishit')).show()

+------------------+----------+
|        avg(ishit)|sum(ishit)|
+------------------+----------+
|0.1103232840173272|     83383|
+------------------+----------+



In [14]:
df2 =sqlsc.read.load('/content/drive/My Drive/Colab Notebooks/big-data-Colab/ad-clicks.csv.gz',
                          format='com.databricks.spark.csv',
                          header = 'true',
                          inferSchema='true')

In [15]:
df2.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- txId: integer (nullable = true)
 |-- userSessionId: integer (nullable = true)
 |-- teamId: integer (nullable = true)
 |-- userId: integer (nullable = true)
 |-- adId: integer (nullable = true)
 |-- adCategory: string (nullable = true)



In [16]:
merge = df.join(df2, 'userid')

In [17]:
merge.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- clickId: integer (nullable = true)
 |-- userSessionId: integer (nullable = true)
 |-- isHit: integer (nullable = true)
 |-- teamId: integer (nullable = true)
 |-- teamLevel: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- txId: integer (nullable = true)
 |-- userSessionId: integer (nullable = true)
 |-- teamId: integer (nullable = true)
 |-- adId: integer (nullable = true)
 |-- adCategory: string (nullable = true)



In [18]:
merge.show(5)

+------+-------------------+-------+-------------+-----+------+---------+-------------------+-----+-------------+------+----+-----------+
|userId|          timestamp|clickId|userSessionId|isHit|teamId|teamLevel|          timestamp| txId|userSessionId|teamId|adId| adCategory|
+------+-------------------+-------+-------------+-----+------+---------+-------------------+-----+-------------+------+----+-----------+
|  1362|2016-05-26 15:07:20|     22|         5739|    0|    13|        1|2016-06-16 10:21:01|39733|        34223|    13|   1|     sports|
|  1362|2016-05-26 15:07:20|     22|         5739|    0|    13|        1|2016-06-15 23:52:15|38854|        34223|    13|   3|electronics|
|  1362|2016-05-26 15:07:20|     22|         5739|    0|    13|        1|2016-06-15 12:23:31|37940|        34223|    13|  15|     sports|
|  1362|2016-05-26 15:07:20|     22|         5739|    0|    13|        1|2016-06-13 00:12:01|32627|        26427|    13|  14|    fashion|
|  1362|2016-05-26 15:07:20|     2